In [ ]:
import open3d as o3d
import numpy as np
import rasterio
from scipy.spatial import cKDTree

def load_point_cloud_from_tif(tif_path):
    with rasterio.open(tif_path) as dataset:
        elevation = dataset.read(1)
        rows, cols = np.where(elevation != dataset.nodata)
        x_coords = cols
        y_coords = rows
        z_coords = elevation[rows, cols]
        points = np.vstack((x_coords, y_coords, z_coords)).T
        return points

def load_point_cloud_from_ply(ply_path):
    pcd = o3d.io.read_point_cloud(ply_path)
    points = np.asarray(pcd.points)
    return points

def normalize_point_cloud(points):
    centroid = np.mean(points, axis=0)
    points = points - centroid
    scale = np.max(np.linalg.norm(points, axis=1))
    points = points / scale
    return points

def compute_euclidean_distance(gt_points, pred_points):
    tree = cKDTree(pred_points)
    distances, _ = tree.query(gt_points, k=1)
    return distances

gt_points = load_point_cloud_from_tif("../archive/D1/D1/depth/depthT_S02951.tif")
pred_points = load_point_cloud_from_ply("../colmap_workspace/seathru/dense/0/fused.ply")

gt_points_normalized = normalize_point_cloud(gt_points)
pred_points_normalized = normalize_point_cloud(pred_points)
distances = compute_euclidean_distance(gt_points_normalized, pred_points_normalized)

mean_distance = np.mean(distances)
print("Mean Euclidean Distance:", mean_distance)